train classifier 

In [11]:
import csv 
import pandas as pd 

In [12]:
df = pd.read_csv(r'/Users/qiweili/Desktop/other_clickbait/all_data.csv')

In [13]:
data = df.headline
target = df.is_clickbait

In [14]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(\
    data, target, test_size=0.20, random_state=42)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

In [16]:
#what would happen with IDF=true?
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
X_train_tf = tfidf_transformer.fit_transform(X_train_counts)

In [17]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='hinge', penalty='l2'\
    ,alpha=1e-3, random_state=42,max_iter=5, tol=None).fit(X_train_tf, y_train)


In [18]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect',CountVectorizer()),
                     ('tfidf',TfidfTransformer()),
                    ('clf',SGDClassifier()),
                    ])

In [19]:
text_clf.fit(X_train,y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [20]:
import numpy as np 
predicted = text_clf.predict(X_test)

In [21]:
np.mean(predicted==y_test)

0.97078125

In [22]:
from sklearn import metrics
metrics.confusion_matrix(y_test,predicted)

array([[3087,   63],
       [ 124, 3126]])

In [23]:
#get precision recall score
y_score = text_clf.decision_function(X_test)

from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, y_score)

print('Average precision-recall score: {0:0.7f}'.format(\
      average_precision))

Average precision-recall score: 0.9955053


using headlines I collected

In [24]:
from sklearn.metrics import classification_report 
target_names=['clickbait', 'nonclickbait']
classification_report(y_test, predicted, target_names=target_names)

'              precision    recall  f1-score   support\n\n   clickbait       0.96      0.98      0.97      3150\nnonclickbait       0.98      0.96      0.97      3250\n\n   micro avg       0.97      0.97      0.97      6400\n   macro avg       0.97      0.97      0.97      6400\nweighted avg       0.97      0.97      0.97      6400\n'

In [25]:
#loop through the entire folder to read csv 

In [60]:
import glob
import os
path = '/Users/qiweili/Desktop/other_clickbait/full_data'
extension = 'csv'

os.chdir(path)
results = [i for i in glob.glob('*.{}'.format(extension))]


In [62]:
#read every result csv in the file
for result in results: 
    file = pd.read_csv(r'/Users/qiweili/Desktop/other_clickbait/full_data/'+result)
    X_test = file.title
    source_predicted = text_clf.predict(X_test)
    se = pd.Series(source_predicted)
    file['is_clickbait'] = se.values
    file.to_csv(r'/Users/qiweili/Desktop/other_clickbait/classified_headlines_all/'+result,sep=',')
    